# Задача 3: Агенты - Инвестиционный помощник

Пример реализации инвестиционного помощника с использованием langchain.

## Импорт необходимых библиотек

In [ ]:
import os
import pandas as pd
from typing import Optional

# LangChain
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import Tool
from langchain_core.messages import HumanMessage, SystemMessage

# Для работы с CSV
import io
import csv

## Пример данных (CSV с акциями)

In [ ]:
# Пример CSV данных с акциями
stocks_csv = """ticker,price,change_percent
SBER,285.50,2.5
GAZP,165.30,-1.2
YNDX,3789.90,3.1
LKOH,7248.60,-2.8
GMKN,14520.50,1.5
ROSN,485.30,5.2
VTBR,0.312,-0.8
MTSS,425.10,2.3
"""

# Загрузка данных в DataFrame
df_stocks = pd.read_csv(io.StringIO(stocks_csv))
print("Данные об акциях:")
print(df_stocks.to_string(index=False))

## Класс инвестиционного помощника

In [ ]:
class InvestmentAssistant:
    """
    Инвестиционный помощник для анализа акций.
    
    Специализация: анализ акций на основе данных CSV (тикер, цена, изменение %)
    Рекомендации: что купить, что продать, что держать
    """
    
    def __init__(self, llm_model: str = "gpt-3.5-turbo", api_key: Optional[str] = None):
        """
        Инициализация инвестиционного помощника.
        
        Args:
            llm_model: название модели LLM
            api_key: API ключ для LLM (если не задан, берется из переменной окружения)
        """
        # Инициализация LLM
        api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.llm = ChatOpenAI(model=llm_model, temperature=0, api_key=api_key)
        
        # Определение специализации
        self.specialization = "анализ акций и предоставление инвестиционных рекомендаций на основе данных CSV"
        
        # Системный промпт
        self.system_prompt = f"""Ты - профессиональный инвестиционный помощник.
Твоя специализация: {self.specialization}.

Ты анализируешь данные об акциях в формате CSV, которые содержат:
- ticker: тикер акции
- price: текущая цена
- change_percent: изменение цены в процентах за текущий период

На основе этих данных ты предоставляешь рекомендации:
- КУПИТЬ: акции с положительной динамикой и хорошими перспективами
- ПРОДАТЬ: акции с отрицательной динамикой или высокими рисками
- ДЕРЖАТЬ: акции со стабильной динамикой

Твои рекомендации должны быть обоснованными и учитывать:
- Процент изменения цены
- Текущую цену
- Общую динамику рынка

Если запрос не относится к анализу акций или инвестициям, вежливо откажи:
"Извините, но моя задача выполнять {self.specialization}."
"""
    
    def is_in_scope(self, request: str) -> bool:
        """
        Проверяет, относится ли запрос к специализации агента.
        
        Args:
            request: текст запроса
        
        Returns:
            bool: True если запрос в рамках специализации
        """
        # Ключевые слова, связанные с инвестициями и акциями
        investment_keywords = [
            'акции', 'акция', 'инвестиции', 'инвестировать', 'купить', 'продать',
            'держать', 'портфель', 'тикер', 'цена', 'рынок', 'дивиденды',
            'stock', 'invest', 'buy', 'sell', 'hold', 'portfolio', 'ticker',
            'csv', 'данные', 'анализ', 'рекомендация'
        ]
        
        request_lower = request.lower()
        return any(keyword in request_lower for keyword in investment_keywords)
    
    def analyze_stocks(self, csv_data: str) -> str:
        """
        Анализирует данные об акциях из CSV и возвращает рекомендации.
        
        Args:
            csv_data: данные в формате CSV
        
        Returns:
            str: рекомендации по акциям
        """
        # Загрузка данных
        df = pd.read_csv(io.StringIO(csv_data))
        
        # Формирование промпта с данными
        data_summary = df.to_string(index=False)
        
        prompt = f"""Проанализируй следующие данные об акциях:

{data_summary}

Предоставь рекомендации:
1. Что КУПИТЬ (акции с хорошей динамикой)
2. Что ПРОДАТЬ (акции с плохой динамикой)
3. Что ДЕРЖАТЬ (стабильные акции)

Для каждой рекомендации укажи тикер и обоснование.
"""
        
        # Генерация ответа
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=prompt)
        ]
        
        response = self.llm.invoke(messages)
        return response.content
    
    def process_request(self, request: str, csv_data: Optional[str] = None) -> str:
        """
        Обрабатывает запрос пользователя.
        
        Args:
            request: текст запроса
            csv_data: данные CSV (опционально, можно передать в запросе)
        
        Returns:
            str: ответ агента
        """
        # Проверка на соответствие специализации
        if not self.is_in_scope(request):
            return f"Извините, но моя задача выполнять {self.specialization}."
        
        # Если в запросе есть CSV данные, анализируем их
        if csv_data:
            return self.analyze_stocks(csv_data)
        
        # Если CSV данных нет, но запрос в рамках специализации, просим данные
        if 'csv' not in request.lower() and 'данные' not in request.lower():
            return "Для анализа акций мне нужны данные в формате CSV. Пожалуйста, предоставьте CSV файл с колонками: ticker, price, change_percent"
        
        # Генерация общего ответа
        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=request)
        ]
        
        response = self.llm.invoke(messages)
        return response.content

## Пример использования инвестиционного помощника

In [ ]:
# Инициализация помощника
# ВАЖНО: установите переменную окружения OPENAI_API_KEY или передайте api_key
# assistant = InvestmentAssistant(api_key="your-api-key-here")

# Пример 1: Анализ акций
request1 = "Проанализируй эти акции и дай рекомендации"
# response1 = assistant.process_request(request1, csv_data=stocks_csv)
# print("Запрос 1:", request1)
# print("Ответ:", response1)
# print("\n" + "="*50 + "\n")

In [ ]:
# Пример 2: Запрос вне компетенции
request2 = "Напиши стихотворение про кота"
# response2 = assistant.process_request(request2)
# print("Запрос 2:", request2)
# print("Ответ:", response2)
# print("\n" + "="*50 + "\n")

## Пример REST API (FastAPI)

In [ ]:
# Пример реализации REST API
# Для запуска: pip install fastapi uvicorn

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI(title="Investment Assistant API")

# Инициализация помощника (глобально)
# assistant = InvestmentAssistant()

class GenerateRequest(BaseModel):
    request: str
    csv_data: Optional[str] = None

class GenerateResponse(BaseModel):
    response: str
    in_scope: bool

@app.post("/generate", response_model=GenerateResponse)
async def generate(request: GenerateRequest):
    """
    Эндпоинт для генерации ответа агента.
    
    Пример запроса:
    {
        "request": "Проанализируй эти акции",
        "csv_data": "ticker,price,change_percent\nSBER,285.50,2.5"
    }
    """
    try:
        # Обработка запроса
        response_text = assistant.process_request(
            request.request,
            csv_data=request.csv_data
        )
        
        # Проверка, в рамках ли специализации
        in_scope = assistant.is_in_scope(request.request)
        
        return GenerateResponse(
            response=response_text,
            in_scope=in_scope
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Для запуска API:
# uvicorn app:app --reload

## Пример Telegram-бота

In [ ]:
# Пример реализации Telegram-бота
# Для запуска: pip install python-telegram-bot

from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

# Инициализация помощника
# assistant = InvestmentAssistant()

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик команды /start"""
    await update.message.reply_text(
        "Привет! Я инвестиционный помощник. "
        "Пришлите мне CSV файл с данными об акциях (ticker, price, change_percent) "
        "или текстовый запрос для анализа."
    )

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик текстовых сообщений"""
    user_message = update.message.text
    
    # Обработка запроса
    response = assistant.process_request(user_message)
    
    await update.message.reply_text(response)

async def handle_document(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик документов (CSV файлов)"""
    file = await context.bot.get_file(update.message.document.file_id)
    
    # Скачивание файла
    file_content = await file.download_as_bytearray()
    csv_data = file_content.decode('utf-8')
    
    # Анализ данных
    response = assistant.process_request("Проанализируй эти акции", csv_data=csv_data)
    
    await update.message.reply_text(response)

def main():
    """Запуск бота"""
    # Замените на ваш токен бота
    application = Application.builder().token("YOUR_BOT_TOKEN").build()
    
    # Регистрация обработчиков
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_handler(MessageHandler(filters.Document.ALL, handle_document))
    
    # Запуск бота
    application.run_polling()

# if __name__ == '__main__':
#     main()

## Тестирование методом агент-агент

In [ ]:
class AgentEvaluator:
    """
    Агент-оценщик для проверки качества ответов инвестиционного помощника.
    """
    
    def __init__(self, llm_model: str = "gpt-3.5-turbo", api_key: Optional[str] = None):
        api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.llm = ChatOpenAI(model=llm_model, temperature=0, api_key=api_key)
    
    def evaluate_response(self, request: str, expected_response: str, actual_response: str) -> dict:
        """
        Оценивает качество ответа агента.
        
        Args:
            request: исходный запрос
            expected_response: ожидаемый ответ
            actual_response: фактический ответ агента
        
        Returns:
            dict: оценка и комментарий
        """
        evaluation_prompt = f"""Ты - эксперт по оценке качества работы AI-агентов.

Запрос: {request}

Ожидаемый ответ: {expected_response}

Фактический ответ агента: {actual_response}

Оцени качество ответа агента по шкале от 0 до 10, где:
- 10: ответ полностью соответствует ожиданиям и специализации
- 5: ответ частично соответствует, но есть недостатки
- 0: ответ не соответствует ожиданиям или вне специализации

Верни оценку в формате:
Оценка: [число от 0 до 10]
Комментарий: [краткое объяснение оценки]
"""
        
        messages = [HumanMessage(content=evaluation_prompt)]
        response = self.llm.invoke(messages)
        
        # Парсинг ответа
        evaluation_text = response.content
        
        # Извлечение оценки (простой парсинг)
        score = 5  # По умолчанию
        if "Оценка:" in evaluation_text:
            try:
                score_line = [line for line in evaluation_text.split("\n") if "Оценка:" in line][0]
                score = int(score_line.split(":")[1].strip())
            except:
                pass
        
        return {
            "score": score,
            "comment": evaluation_text
        }

# Тестовые кейсы
test_cases = [
    {
        "request": "Проанализируй эти акции: ticker,price,change_percent\nSBER,285.50,2.5\nGAZP,165.30,-1.2",
        "expected": "Должен содержать рекомендации: КУПИТЬ, ПРОДАТЬ, ДЕРЖАТЬ с обоснованием",
        "csv_data": "ticker,price,change_percent\nSBER,285.50,2.5\nGAZP,165.30,-1.2"
    },
    {
        "request": "Напиши стихотворение",
        "expected": "Извините, но моя задача выполнять анализ акций...",
        "csv_data": None
    },
    # TODO: Добавьте еще 8 тестовых кейсов
]

def run_tests(assistant: InvestmentAssistant, evaluator: AgentEvaluator):
    """
    Запускает тестирование агента.
    """
    results = []
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nТест {i}: {test_case['request'][:50]}...")
        
        # Получение ответа агента
        actual_response = assistant.process_request(
            test_case['request'],
            csv_data=test_case.get('csv_data')
        )
        
        # Оценка ответа
        evaluation = evaluator.evaluate_response(
            test_case['request'],
            test_case['expected'],
            actual_response
        )
        
        results.append({
            "test_id": i,
            "request": test_case['request'],
            "expected": test_case['expected'],
            "actual": actual_response,
            "score": evaluation['score'],
            "comment": evaluation['comment']
        })
        
        print(f"Оценка: {evaluation['score']}/10")
    
    # Вывод результатов
    print("\n" + "="*50)
    print("РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ")
    print("="*50)
    
    total_score = sum(r['score'] for r in results)
    avg_score = total_score / len(results)
    
    print(f"\nОбщая оценка: {total_score}/{len(results) * 10}")
    print(f"Средняя оценка: {avg_score:.2f}/10")
    
    for result in results:
        print(f"\nТест {result['test_id']}:")
        print(f"  Запрос: {result['request'][:50]}...")
        print(f"  Оценка: {result['score']}/10")
        print(f"  Комментарий: {result['comment'][:100]}...")
    
    return results

# Пример запуска тестов
# evaluator = AgentEvaluator()
# results = run_tests(assistant, evaluator)